In [51]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [107]:
# # Importing the dataset
data = pd.read_csv('s1_improvised_only_grades.csv')
data= data.drop('NO.',axis=1)

# users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
# ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')


In [108]:
# Preparing the training set and the test set
training_set = pd.read_csv('s1_train.csv')
training_set= training_set.drop('NO.',axis=1)
training_set = np.array(training_set, dtype = 'int')#user_id,movie_id,ratings
test_set = pd.read_csv('s1_test.csv')
test_set= test_set.drop('NO.',axis=1)
test_set = np.array(test_set, dtype = 'int')
test_set.shape
#training_set.shape

(11, 9)

In [109]:
test_set.shape
#training_set.shape

(11, 9)

In [110]:
#nb_students = int(max(max(training_set[:,0]), max(test_set[:,0])))
#nb_courses = int(max(max(training_set[:,1]), max(test_set[:,1])))
nb_students = max(training_set.shape[0], test_set.shape[0])-1
nb_courses = max(training_set.shape[1], test_set.shape[1])
nb_students
#nb_courses

42

In [111]:
# # Converting the data into an array with users in lines and movies in columns
# def convert(data):
#     new_data = []
#     for id_users in range(1, nb_users + 1):
#         id_movies = data[:,1][data[:,0] == id_users]
#         id_ratings = data[:,2][data[:,0] == id_users]
#         ratings = np.zeros(nb_movies)
#         ratings[id_movies - 1] = id_ratings
#         new_data.append(list(ratings))
#     return new_data
# training_set = convert(training_set)
# test_set = convert(test_set)
# training_set= training_set.drop('NO',axis=1)
# test_set= training_set.drop('NO',axis=1)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [112]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_courses, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_courses)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x.view(x.size(0), -1) 
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)


In [113]:
# # Training the SAE
# nb_epoch = 200
# for epoch in range(1, nb_epoch + 1):
#     train_loss = 0
#     s = 0.
#     for id_user in range(nb_users):
#         input = Variable(training_set[id_user]).unsqueeze(0)
#         target = input.clone()
#         if torch.sum(target.data > 0) > 0:
#             output = sae(input)
#             target = Variable(test_set[id_user]).unsqueeze(0)
#             output[target == 0] = 0
#             loss = criterion(output, target)
#             mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
#             loss.backward()
#             train_loss += np.sqrt(loss.data*mean_corrector)
#             s += 1.
#             optimizer.step()
#     print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


In [114]:
# # Testing the SAE
# test_loss = 0
# s = 0.
# for id_user in range(nb_users):
#     input = Variable(training_set[id_user]).unsqueeze(0)
#     #target = Variable(test_set[id_user])
#     target = Variable(test_set[id_user]).unsqueeze(0)
#     if torch.sum(target.data > 0) > 0:
#         output = sae(input)
#         target.require_grad = False
#         output[(target == 0).unsqueeze(0)] = 0
#         loss = criterion(output, target)
#         mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
#         test_loss += np.sqrt(loss.data*mean_corrector)
#         s += 1.
# print('test loss: '+str(test_loss/s))

In [115]:
# # Training the SAE
# nb_epoch = 200
# for epoch in range(1, nb_epoch + 1):
#     train_loss = 0
#     s = 0.
#     for id_user in range(nb_students):
#         input = Variable(training_set[id_user]).unsqueeze(0)
#         target = input.clone()
    
#         if torch.sum(target.data > 0) > 0:
#             output = sae(input.unsqueeze_(2))
#             print(input.shape)
#             target.require_grad = False
#             output[target == 0] = 0
#             loss = criterion(output, target)
#             mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
#             loss.backward()
#             #train_loss += np.sqrt(loss.data[0]*mean_corrector)
#             train_loss += np.sqrt(loss.item()*mean_corrector)

#             s += 1.
#             optimizer.step()
#     print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
# # data = data.view(data.size(0), -1)
# # output = model(data)[]

In [116]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_students):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            
            train_loss += np.sqrt(loss.item()*mean_corrector)

            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: 2.667836700131636
epoch: 2 loss: 2.0094775790842117
epoch: 3 loss: 1.973593640106903
epoch: 4 loss: 1.9545548520625307
epoch: 5 loss: 2.0921537637861567
epoch: 6 loss: 2.2826654946161575
epoch: 7 loss: 2.12166120505108
epoch: 8 loss: 1.7192527496096939
epoch: 9 loss: 1.93712849460893
epoch: 10 loss: 1.88637620735018
epoch: 11 loss: 1.9746038734889761
epoch: 12 loss: 1.7509764883229686
epoch: 13 loss: 1.605379937247395
epoch: 14 loss: 1.6935663302931097
epoch: 15 loss: 1.7068470101630264
epoch: 16 loss: 1.9402216364876819
epoch: 17 loss: 1.8702705514303892
epoch: 18 loss: 1.6368472966619105
epoch: 19 loss: 1.7890178563606949
epoch: 20 loss: 1.6608327905236198
epoch: 21 loss: 1.671960764138635
epoch: 22 loss: 1.9513138647507755
epoch: 23 loss: 1.883091554445884
epoch: 24 loss: 1.6976882193813727
epoch: 25 loss: 1.9066723084332884
epoch: 26 loss: 1.7744466530485057
epoch: 27 loss: 1.631819798843946
epoch: 28 loss: 1.7298206660713884
epoch: 29 loss: 1.7904319879858548
epoch:

In [117]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_students):
    input = Variable(training_set[id_user]).unsqueeze(0)
    #target = Variable(test_set[id_user]).unsqueeze(0)
for id_user in range(test_set.shape[0]):
    target = Variable(test_set[id_user]).unsqueeze(0)


    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)

        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 1.4992164448274343


In [118]:
# user_id = 1
# movie_title = movies.iloc[:nb_courses, 1:2]
# print(movie_title)
# user_rating = training_set.data.numpy()[user_id, :].reshape(-1,1)
# user_target = test_set.data.numpy()[user_id, :].reshape(-1,1)
 
# user_input = Variable(training_set[user_id]).unsqueeze(0)
# predicted = sae(user_input)
# predicted = predicted.data.numpy().reshape(-1,1)
 
# # Join all info in one dataset
# result_array = np.hstack([movie_title, user_target, predicted])
# result_array = result_array[result_array[:, 1] > 0]
# result_df = pd.DataFrame(data=result_array, columns=['Movie', 'Target Rating', 'Predicted'])

In [119]:
test_set

tensor([[ 0.,  0.,  5.,  7.,  6.,  5.,  7.,  9.,  8.],
        [ 0.,  4.,  0.,  7.,  6.,  0., 10.,  9.,  8.],
        [ 8.,  8.,  7.,  9.,  7.,  7., 10., 10., 10.],
        [ 0.,  0.,  7.,  0.,  7.,  0., 10.,  7., 10.],
        [ 6.,  9.,  8.,  7.,  7.,  0., 10., 10.,  8.],
        [10., 10.,  6.,  8.,  9.,  8., 10.,  9., 10.],
        [ 7.,  7.,  5.,  7.,  7.,  5., 10.,  9., 10.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  9.,  8.,  7.],
        [ 7.,  7.,  6.,  7.,  8.,  7., 10., 10., 10.],
        [ 6.,  7.,  8.,  6.,  7.,  6., 10., 10., 10.],
        [10.,  7.,  0.,  6.,  7.,  6., 10., 10., 10.]])

In [120]:
columns=data.columns
c=columns.tolist()
d=pd.DataFrame(c)
d

,0
0,MA101
1,PH100
2,BE10105
3,BE110
4,BE103
5,EC100
6,PH110
7,EC110
8,CS110


In [121]:
student_id = 4
print(movie_title.shape)
student_grade = training_set.data.numpy()[student_id, :].reshape(-1,1)
user_target = test_set.data.numpy()[student_id, :].reshape(-1,1)
 
user_input = Variable(training_set[student_id]).unsqueeze(0)
predicted = sae(user_input)
predicted = predicted.data.numpy().reshape(-1,1)
 
# Join all info in one dataset
result_array = np.hstack([d, user_target, predicted])
result_array = result_array[result_array[:, 1] > 0]
result_df = pd.DataFrame(data=result_array, columns=['courses', 'Target Rating', 'Predicted'])

(9, 1)


In [122]:

result_df


,courses,Target Rating,Predicted
0,MA101,6,6.30541
1,PH100,9,6.65035
2,BE10105,8,6.50106
3,BE110,7,7.73212
4,BE103,7,7.14605
5,PH110,10,8.15488
6,EC110,10,10.1389
7,CS110,8,10.5107


In [73]:
user_rating

array([[ 6.],
       [ 6.],
       [ 5.],
       [ 6.],
       [ 6.],
       [ 0.],
       [10.],
       [ 8.],
       [10.]], dtype=float32)

In [74]:
training_set

tensor([[ 6.,  6.,  5.,  6.,  6.,  0., 10.,  8., 10.],
        [ 5.,  6.,  5.,  6.,  7.,  0., 10.,  9.,  9.],
        [ 0.,  0.,  0.,  0.,  7.,  0.,  9.,  7.,  8.],
        [10.,  8.,  0., 10.,  8.,  7., 10., 10.,  9.],
        [ 0.,  0.,  9.,  6.,  6.,  0., 10.,  8.,  9.],
        [ 7.,  7.,  8.,  8.,  6.,  0., 10., 10., 10.],
        [ 6.,  6.,  5.,  0.,  6.,  5., 10., 10., 10.],
        [ 7.,  9.,  8., 10.,  8.,  6., 10., 10., 10.],
        [ 5.,  5.,  6.,  7.,  7.,  6.,  9., 10.,  9.],
        [ 7.,  6.,  5.,  9.,  7.,  6., 10.,  9.,  9.],
        [ 5.,  6.,  5.,  7.,  6.,  0., 10., 10.,  8.],
        [10.,  8.,  7.,  9.,  6.,  7.,  9.,  8., 10.],
        [ 5.,  6.,  0.,  7.,  6.,  5., 10., 10.,  9.],
        [ 8.,  7.,  6.,  7.,  7.,  7., 10., 10., 10.],
        [ 7.,  7.,  6., 10.,  7.,  7.,  9.,  8., 10.],
        [ 9.,  9.,  8.,  8.,  8.,  9., 10., 10., 10.],
        [ 9.,  7.,  0.,  8.,  7.,  9., 10., 10.,  9.],
        [ 7.,  6.,  9., 10.,  8.,  6.,  9., 10., 10.],
        [ 

In [75]:
user_target

array([[0.],
       [0.],
       [5.],
       [7.],
       [6.],
       [5.],
       [7.],
       [9.],
       [8.]], dtype=float32)

In [76]:
user_input

tensor([[ 6.,  6.,  5.,  6.,  6.,  0., 10.,  8., 10.]])

In [77]:
predicted

array([[ 7.13807  ],
       [ 5.618593 ],
       [ 7.492981 ],
       [ 7.364008 ],
       [ 6.6475687],
       [ 7.511931 ],
       [ 9.864161 ],
       [ 7.822873 ],
       [10.628405 ]], dtype=float32)

In [18]:
m=movies.iloc[0,:]
print(m.shape)

(10,)


In [ ]:
m

In [ ]:
movie_title[:nb_courses]

In [ ]:
nb_courses

In [21]:
index= movies.index

In [90]:
columns=data.columns
c=columns.tolist()
d=pd.DataFrame(c)

In [92]:
d

,0
0,MA101
1,PH100
2,BE10105
3,BE110
4,BE103
5,EC100
6,PH110
7,EC110
8,CS110
